In [1]:
# Cell 1

!pip install pytorch-lightning==2.0.2 transformers datasets evaluate sacrebleu sentencepiece wandb --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Cell 2

import torch
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader, Sampler
from torch.optim import AdamW

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
import evaluate
import wandb
from pytorch_lightning.loggers import WandbLogger

# (Optional) If your environment requires explicit login:
wandb.login()


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: f20213032 (f20213032-bits-pilani) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
# Cell 3

import random

class RandomSubsetSampler(Sampler):
    """
    Samples a random subset of the dataset each epoch.
    Each epoch, we pick 'subset_size' distinct random indices from [0 .. dataset_size).
    If you're on single GPU, you might need to manually call set_epoch each epoch if
    it doesn't automatically update. On distributed training, Lightning handles it.
    """
    def __init__(self, dataset_size, subset_size, seed=42):
        self.dataset_size = dataset_size
        self.subset_size = subset_size
        self.base_seed = seed
        self.epoch = 0

    def __iter__(self):
        # Re-seed each epoch for a different subset
        random.seed(self.base_seed + self.epoch)
        indices = random.sample(range(self.dataset_size), self.subset_size)
        return iter(indices)

    def __len__(self):
        return self.subset_size

    def set_epoch(self, epoch: int):
        self.epoch = epoch


In [4]:
# Cell 4

print("Loading the IITB English-Hindi dataset...")
dataset = load_dataset("cfilt/iitb-english-hindi")
print(dataset)

# Optional: Inspect a sample
# print(dataset["train"][0])


Loading the IITB English-Hindi dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.14k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/190M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/500k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})


In [5]:
print(dataset["train"][0])


{'translation': {'en': 'Give your application an accessibility workout', 'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}}


In [6]:
# Cell 5

def compute_token_accuracy(decoded_preds, decoded_labels):
    """
    Token-level accuracy:
    - For each pred/ref pair, we split on whitespace.
    - Compare up to min(len(pred), len(ref)).
    - Count how many tokens match exactly.
    - Then sum across all examples.
    """
    total_correct = 0
    total_tokens = 0
    for pred, ref in zip(decoded_preds, decoded_labels):
        pred_tokens = pred.strip().split()
        ref_tokens = ref.strip().split()
        limit = min(len(pred_tokens), len(ref_tokens))
        for i in range(limit):
            if pred_tokens[i] == ref_tokens[i]:
                total_correct += 1
        total_tokens += limit
    return total_correct / total_tokens if total_tokens else 0.0


In [7]:
# Cell 6

model_name = "Helsinki-NLP/opus-mt-en-hi"  # MarianMT (English->Hindi)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    # 'examples["translation"]' is a dict with {'en', 'hi'}
    en_texts = [row["en"] for row in examples["translation"]]
    hi_texts = [row["hi"] for row in examples["translation"]]
    return tokenizer(
        en_texts,
        text_target=hi_texts,
        max_length=128,
        truncation=True
    )

print("Tokenizing dataset ...")
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["translation"]
)


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Tokenizing dataset ...


Map:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Map:   0%|          | 0/520 [00:00<?, ? examples/s]

Map:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [8]:
# Cell 7

from transformers import DataCollatorForSeq2Seq

class TranslationDataModule(pl.LightningDataModule):
    def __init__(
        self,
        tokenized_dataset,
        tokenizer,
        batch_size=16,
        subset_size=50000,
        seed=42
    ):
        super().__init__()
        self.tokenized_dataset = tokenized_dataset
        self.batch_size = batch_size
        self.subset_size = subset_size
        self.seed = seed

        # Collator handles dynamic padding
        self.data_collator = DataCollatorForSeq2Seq(
            tokenizer=tokenizer,
            model=None,
            padding=True
        )

    def setup(self, stage=None):
        self.train_data = self.tokenized_dataset["train"]
        self.val_data   = self.tokenized_dataset["validation"]
        self.test_data  = self.tokenized_dataset["test"]

    def train_dataloader(self):
        # We'll use the RandomSubsetSampler for the train set
        sampler = RandomSubsetSampler(
            dataset_size=len(self.train_data),
            subset_size=self.subset_size,
            seed=self.seed
        )
        return DataLoader(
            self.train_data,
            batch_size=self.batch_size,
            sampler=sampler,
            collate_fn=self.data_collator
        )

    def val_dataloader(self):
        # Typically use the entire validation set
        return DataLoader(
            self.val_data,
            batch_size=self.batch_size,
            collate_fn=self.data_collator
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_data,
            batch_size=self.batch_size,
            collate_fn=self.data_collator
        )

# Create the data module with subset_size=30000
data_module = TranslationDataModule(
    tokenized_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    batch_size=16,
    subset_size=50000,  # each epoch sees 50k random samples from train
    seed=42
)


In [9]:
# Cell 8
!pip install rouge_score nltk -q

bleu_metric = evaluate.load("sacrebleu")
rouge_metric = evaluate.load("rouge")
meteor_metric = evaluate.load("meteor")

class TranslationModel(pl.LightningModule):
    def __init__(self, model_name, lr=1e-4):
        """
        We set an initial LR (1e-4). The LR finder can adjust it if we do auto_lr_find.
        """
        super().__init__()
        self.save_hyperparameters()

        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.lr = lr

    def forward(self, input_ids, attention_mask, labels=None):
        return self.model(input_ids, attention_mask=attention_mask, labels=labels)

    def training_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        # Generate predictions
        preds = self.model.generate(batch["input_ids"], attention_mask=batch["attention_mask"], max_length=128)
        decoded_preds = self.tokenizer.batch_decode(preds, skip_special_tokens=True)

        labels = batch["labels"]
        labels = torch.where(labels != -100, labels, torch.tensor(self.tokenizer.pad_token_id).to(labels.device))
        decoded_labels = self.tokenizer.batch_decode(labels, skip_special_tokens=True)

        # Evaluate BLEU, ROUGE, METEOR
        bleu = bleu_metric.compute(predictions=decoded_preds, references=[[lbl] for lbl in decoded_labels])
        rouge = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)
        meteor = meteor_metric.compute(predictions=decoded_preds, references=decoded_labels)

        # Token-level accuracy
        token_acc = compute_token_accuracy(decoded_preds, decoded_labels)

        self.log("val_loss", loss, prog_bar=True)
        self.log("val_bleu", bleu["score"], prog_bar=True)
        self.log("val_rouge1", rouge["rouge1"])
        self.log("val_rouge2", rouge["rouge2"])
        self.log("val_rougeL", rouge["rougeL"])
        self.log("val_meteor", meteor["meteor"])
        self.log("val_token_accuracy", token_acc, prog_bar=False)
        return loss

    def test_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        preds = self.model.generate(batch["input_ids"], attention_mask=batch["attention_mask"], max_length=128)
        decoded_preds = self.tokenizer.batch_decode(preds, skip_special_tokens=True)

        labels = batch["labels"]
        labels = torch.where(labels != -100, labels, torch.tensor(self.tokenizer.pad_token_id).to(labels.device))
        decoded_labels = self.tokenizer.batch_decode(labels, skip_special_tokens=True)

        bleu = bleu_metric.compute(predictions=decoded_preds, references=[[lbl] for lbl in decoded_labels])
        rouge = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)
        meteor = meteor_metric.compute(predictions=decoded_preds, references=decoded_labels)
        token_acc = compute_token_accuracy(decoded_preds, decoded_labels)

        self.log("test_loss", loss, prog_bar=True)
        self.log("test_bleu", bleu["score"], prog_bar=True)
        self.log("test_rouge1", rouge["rouge1"])
        self.log("test_rouge2", rouge["rouge2"])
        self.log("test_rougeL", rouge["rougeL"])
        self.log("test_meteor", meteor["meteor"])
        self.log("test_token_accuracy", token_acc, prog_bar=False)
        return loss

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=self.lr)


  Preparing metadata (setup.py) ... done


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [10]:
# Cell 9

from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

# Save only the best checkpoint based on val_loss
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    dirpath="./checkpoints",
    filename="model-{epoch:02d}-{val_loss:.2f}",
    save_top_k=1,
    mode="min"
)

# Early stop if val_loss doesn't improve for 3 epochs
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    patience=3,
    mode="min"
)


In [17]:
# Cell 10

print("Evaluating the ORIGINAL model (no fine-tuning) on the test set...")
wandb_logger = WandbLogger(
    project="my-subset-auto-lr-batch-project",
    name="No-tuning-run"
)
# Create a minimal trainer just for testing
trainer_orig = pl.Trainer(
    accelerator="gpu",
    devices=1,
    precision=16,
    max_epochs=0,  # we won't train, just test
    log_every_n_steps=10,
    logger=wandb_logger,  # no need to log
    enable_checkpointing=False
)

# Model with the original pretrained weights
model_orig = TranslationModel(model_name=model_name, lr=1e-4)  # lr won't matter since no training

original_metrics = trainer_orig.test(model_orig, datamodule=data_module)
print("Original (no fine-tuning) Test Metrics:", original_metrics)


INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Evaluating the ORIGINAL model (no fine-tuning) on the test set...


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_bleu         │     9.330301284790039     │
│         test_loss         │    3.9272029399871826     │
│        test_meteor        │    0.29531525292165794    │
│        test_rouge1        │    0.11369776328199195    │
│        test_rouge2        │   0.022937362686066314    │
│        test_rougeL        │    0.11275715356595344    │
│    test_token_accuracy    │    0.07673494517803192    │
└───────────────────────────┴───────────────────────────┘

Original (no fine-tuning) Test Metrics: [{'test_loss': 3.9272029399871826, 'test_bleu': 9.330301284790039, 'test_rouge1': 0.11369776328199195, 'test_rouge2': 0.022937362686066314, 'test_rougeL': 0.11275715356595344, 'test_meteor': 0.29531525292165794, 'test_token_accuracy': 0.07673494517803192}]


In [18]:
# Cell 11

model_ft = TranslationModel(model_name=model_name, lr=1e-4)

wandb_logger = WandbLogger(
    project="my-subset-auto-lr-batch-project",
    name="fine-tuning-run"
)

trainer_ft = pl.Trainer(
    accelerator="gpu",
    devices=1,
    precision=16,
    max_epochs=10,
    log_every_n_steps=10,
    logger=wandb_logger,
    callbacks=[checkpoint_callback, early_stop_callback]
)

# Use the Tuner for auto_lr_find and auto_scale_batch_size
tuner = pl.tuner.Tuner(trainer_ft) # Create a Tuner instance

print("\n>>> Tuning for LR and Batch Size first (on subset of train data)...")
tuner_results = tuner.lr_find(model_ft, datamodule=data_module, mode="exponential", early_stop_threshold=10) # Use tuner.lr_find()
# tuner_results = tuner.scale_batch_size(model_ft, datamodule=data_module, mode="power")  # Use tuner.scale_batch_size() for batch size tuning if needed

# Update the model's learning rate with the suggested value
model_ft.lr = tuner_results.suggestion()

print("Tune results:", tuner_results)
print("Suggested LR:", model_ft.lr)
print("Suggested batch size:", data_module.batch_size) # or tuner_results.scale_batch_size if you used scale_batch_size

print("\n>>> Now fitting with the new hyperparameters...")
trainer_ft.fit(model_ft, datamodule=data_module)

print("\n>>> Done training. Testing the fine-tuned model on the test set...")
fine_tuned_metrics = trainer_ft.test(model_ft, datamodule=data_module)
print("Fine-Tuned Model Test Metrics:", fine_tuned_metrics)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



>>> Tuning for LR and Batch Size first (on subset of train data)...


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=100` reached.
INFO:pytorch_lightning.tuner.lr_finder:Learning rate set to 7.585775750291837e-08
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/.lr_find_cc0c1e7d-8134-40cb-b670-6369ea27296f.ckpt
INFO:pytorch_lightning.utilities.rank_zero:Restored all states from the checkpoint at /content/.lr_find_cc0c1e7d-8134-40cb-b670-6369ea27296f.ckpt


Tune results: <pytorch_lightning.tuner.lr_finder._LRFinder object at 0x784c6055d490>
Suggested LR: 7.585775750291837e-08
Suggested batch size: 16

>>> Now fitting with the new hyperparameters...


...

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



>>> Done training. Testing the fine-tuned model on the test set...


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_bleu         │    10.647638320922852     │
│         test_loss         │    3.6486575603485107     │
│        test_meteor        │    0.3243692357594135     │
│        test_rouge1        │    0.11455275955029587    │
│        test_rouge2        │   0.024601883955693286    │
│        test_rougeL        │    0.11288895034407599    │
│    test_token_accuracy    │    0.08877011388540268    │
└───────────────────────────┴───────────────────────────┘

Fine-Tuned Model Test Metrics: [{'test_loss': 3.6486575603485107, 'test_bleu': 10.647638320922852, 'test_rouge1': 0.11455275955029587, 'test_rouge2': 0.024601883955693286, 'test_rougeL': 0.11288895034407599, 'test_meteor': 0.3243692357594135, 'test_token_accuracy': 0.08877011388540268}]


In [19]:
# Cell 12

print("\n=== COMPARISON OF ORIGINAL vs. FINE-TUNED ===")
print("Original model metrics:", original_metrics)
print("Fine-tuned model metrics:", fine_tuned_metrics)



=== COMPARISON OF ORIGINAL vs. FINE-TUNED ===
Original model metrics: [{'test_loss': 3.9272029399871826, 'test_bleu': 9.330301284790039, 'test_rouge1': 0.11369776328199195, 'test_rouge2': 0.022937362686066314, 'test_rougeL': 0.11275715356595344, 'test_meteor': 0.29531525292165794, 'test_token_accuracy': 0.07673494517803192}]
Fine-tuned model metrics: [{'test_loss': 3.6486575603485107, 'test_bleu': 10.647638320922852, 'test_rouge1': 0.11455275955029587, 'test_rouge2': 0.024601883955693286, 'test_rougeL': 0.11288895034407599, 'test_meteor': 0.3243692357594135, 'test_token_accuracy': 0.08877011388540268}]


In [20]:
# Cell 13

sample_texts = [
    "Hello, how are you?",
    "This is a machine translation test.",
    "Please write your name.",
]

print("\n=== Sample translations from the fine-tuned model ===")
inputs = model_ft.tokenizer(sample_texts, return_tensors="pt", padding=True, truncation=True)
inputs = {k: v.to(model_ft.device) for k, v in inputs.items()}

with torch.no_grad():
    generated_tokens = model_ft.model.generate(**inputs, max_length=40)

translations = model_ft.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
for en, hi in zip(sample_texts, translations):
    print(f"EN: {en}\nHI: {hi}\n")



=== Sample translations from the fine-tuned model ===
EN: Hello, how are you?
HI: हैलो, तुम कैसे हो?

EN: This is a machine translation test.
HI: यह एक मशीन अनुवाद परीक्षण है।

EN: Please write your name.
HI: कृपया अपना नाम लिखें।



In [ ]:
# Cell 14

import os

save_dir = "finetuned_en_hi_model_auto"
os.makedirs(save_dir, exist_ok=True)

model_ft.model.save_pretrained(save_dir)
model_ft.tokenizer.save_pretrained(save_dir)
print(f"Fine-tuned model and tokenizer saved to '{save_dir}'")


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Fine-tuned model and tokenizer saved to 'finetuned_en_hi_model_auto'


In [22]:
!zip -r ./checkpoints.zip ./checkpoints/


  adding: checkpoints/ (stored 0%)
  adding: checkpoints/model-epoch=09-val_loss=4.06.ckpt (deflated 8%)


In [23]:
!zip -r ./wandb.zip ./wandb/
!zip -r ./finetune.zip ./finetuned_en_hi_model_auto/


  adding: wandb/ (stored 0%)
  adding: wandb/run-20250402_155405-1wkbeifm/ (stored 0%)
  adding: wandb/run-20250402_155405-1wkbeifm/logs/ (stored 0%)
  adding: wandb/run-20250402_155405-1wkbeifm/logs/debug-core.log (deflated 57%)
  adding: wandb/run-20250402_155405-1wkbeifm/logs/debug-internal.log (deflated 80%)
  adding: wandb/run-20250402_155405-1wkbeifm/logs/debug.log (deflated 82%)
  adding: wandb/run-20250402_155405-1wkbeifm/run-1wkbeifm.wandb (deflated 80%)
  adding: wandb/run-20250402_155405-1wkbeifm/tmp/ (stored 0%)
  adding: wandb/run-20250402_155405-1wkbeifm/tmp/code/ (stored 0%)
  adding: wandb/run-20250402_155405-1wkbeifm/files/ (stored 0%)
  adding: wandb/run-20250402_155405-1wkbeifm/files/wandb-metadata.json (deflated 43%)
  adding: wandb/run-20250402_155405-1wkbeifm/files/output.log (deflated 74%)
  adding: wandb/run-20250402_155405-1wkbeifm/files/requirements.txt (deflated 55%)
  adding: wandb/latest-run/ (stored 0%)
  adding: wandb/latest-run/logs/ (stored 0%)
  adding

In [24]:
from google.colab import files
files.download('checkpoints.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>